In [ ]:
"""
            *****************************************************************************
            *                         >> IN THE NAME OF ALLAH <<                        *
            *  FINANCIAL INCOME STATEMENT OPTIMIZATION BASED ON UNCERTAINTY CONDITION   *
            *        FINDING BEST FITTED PESSIMISTIC AND OPTIMISTIC REVENUE             *
            *        USING MONTE-CARLO METHOD WITH BETA PROBABILTY FUNCTION             *
            *---------------------------------------------------------------------------*
            *              This program is written by Salar Delavar Qashqai             *
            *                     E-mail:salar.d.ghashghaei@gmail.com                   *
            *****************************************************************************
"""
# Import numpy and plotly
import numpy as np
import time

In [ ]:
def HISTOGRAM_BOXPLOT_PLOTLY( DATA, XLABEL='X', TITLE='A', COLOR='cyan'):
    # Plotting histogram and boxplot
    import plotly.express as px
    fig = px.histogram(x=DATA, marginal="box", color_discrete_sequence=[COLOR])
    fig.update_layout(title=TITLE, xaxis_title=XLABEL, yaxis_title="Frequency")
    fig.show()
    #fig = px.ecdf(irr, title=TITLE)
    #fig.show()

def BETA_PDF(MIN_X, MAX_X, a, b, n):
    import numpy as np
    return MIN_X + (MAX_X - MIN_X) * np.random.beta(a, b, size=n)

def INCOME_STATEMENT(Xmin, Xmax, SIZE_PDF):
    # Define the input variables and their distributions
    Revenue = BETA_PDF(Xmin, Xmax, a=1, b=2, n=SIZE_PDF) # Revenue follows beta(1, 2)
    Direct_Material_Cost = BETA_PDF(100, 200, a=2, b=1, n=SIZE_PDF) # Direct material cost follows beta(2, 1)
    Direct_Labor_Cost = BETA_PDF(10, 20, a=2, b=1, n=SIZE_PDF) # Direct labor cost follows beta(2, 1)
    Overhead_Cost = BETA_PDF(10, 50, a=2, b=1, n=SIZE_PDF) # Overhead cost follows beta(2, 1)
    cost_of_goods_sold = Direct_Material_Cost + Direct_Labor_Cost + Overhead_Cost # Cost of goods sold is the sum of direct material cost, direct labor cost and overhead cost
    gross_profit = Revenue - cost_of_goods_sold # Gross profit is revenue minus cost of goods sold
    operating_expenses = BETA_PDF(5, 10, a=1, b=2, n=SIZE_PDF) # Operating expenses follow beta(1, 2)
    operating_income = gross_profit - operating_expenses # Operating income is gross profit minus operating expenses
    interest_expense = BETA_PDF(5, 10, a=2, b=1, n=SIZE_PDF) # Interest expense follows beta(2 ,1)
    income_before_tax = operating_income - interest_expense # Income before tax is operating income minus interest expense
    income_tax_expense = income_before_tax * 0.3 # Income tax expense is 30% of income before tax
    net_income = income_before_tax - income_tax_expense # Net income is income before tax minus income tax expense

    # Calculate the output variable (net income margin) for each sample
    net_income_margin = 100 * net_income / Revenue # Net income margin is net income divided by revenue
    return net_income_margin

def OPTIMIZATION_REVENUE(NIM_MIN, NIM_MAX, SIZE_PDF, SIZE_MESH, MIIG, MAIG):
    Xmin_grid = np.linspace(0, MIIG, SIZE_MESH) # Minimum revenue
    Xmax_grid = np.linspace(MIIG, MAIG, SIZE_MESH) # Maximum revenue

    # Initialize the best pair and the minimum difference
    best_pair = (None, None) # Best pair of Xmin and Xmax
    min_diff = np.inf # Minimum difference between actual and desired percentiles
    it = 0 # Iteration counts
    STOP = 0 # Iteration Stop
    # monitor cpu time
    starttime = time.process_time()
    # Loop over the grid and evaluate the formula for each pair
    for Xmin in Xmin_grid:
        for Xmax in Xmax_grid:
            # Calculate the NIM using the formula
            Z = INCOME_STATEMENT(Xmin, Xmax, SIZE_PDF)
            # Calculate the actual quantiles
            actual_Q1 = np.quantile(Z, 0.25)
            actual_Q3 = np.quantile(Z, 0.75)
            # Calculate the difference between actual and desired quantiles
            diff = np.abs(actual_Q1 - NIM_MIN) + np.abs(actual_Q3 - NIM_MAX)
            # Update the best pair and the minimum difference if the current pair is better
            it += 1;
            if actual_Q1 >= NIM_MIN and actual_Q3 <= NIM_MAX:
                print(f"\n\t\t Feasible Solution Found with {it} Iterations - diff: {diff:.4f}")
                print(f"\t\t The best pair of Xmin: {Xmin:.2f} and Xmax: {Xmax:.2f}")
                print(f"\t\t Q1: {actual_Q1:.2f} - Q3: {actual_Q3:.2f}")
                totaltime = time.process_time() - starttime
                print(f'\t\t Iterations Total time (s): {totaltime:.2f} \n\n')
                STOP = 1
                break
        if STOP == 1:
            break
                
    # Control for Feasible Solution
    if it == (SIZE_MESH * SIZE_MESH):
        STOP = 0; Xmin = 0; Xmax = 0;
        print("\n Feasible Solution Not Found - Change Xmin_grid and Xmax_grid or Increase SIZE_PDF or SIZE_MESH and try again")
        print(f"\n The minimum difference between actual and desired percentiles is: {diff:.2f}")
        Q25_Z = np.quantile(Z, 0.25); Q75_Z = np.quantile(Z, 0.75);
        print(f" Net Income Margin (Q25): {Q25_Z:.2f}"); print(f" Net Income Margin (Q75): {Q75_Z:.2f}");
        
    return STOP, Xmin, Xmax    

In [ ]:
### find optimum Xmin and Xmax  when quantile(Net Income Margin , .25) <= 5 and quantile(Net Income Margin , .75) >= 30

# Define the desired percentiles
NIM_MIN = 5 # MIN: 25th percentile
NIM_MAX = 30 # MAX: 75th percentile
SIZE_PDF = 10000 # Number of samples

# Define the grid of possible values for Xmin and Xmax
SIZE_MESH = 100 # Number of mesh samples
MIIG = 200 # MIN REVENUE INTIAL GUESS 
MAIG = 800 # MAX REVENUE INTIAL GUESS

stop, Xmin, Xmax = OPTIMIZATION_REVENUE(NIM_MIN, NIM_MAX, SIZE_PDF, SIZE_MESH, MIIG, MAIG)

# Calculate Optimum Revenue
REVENUE_OPTIMUM = BETA_PDF(Xmin, Xmax, a=1, b=2, n=SIZE_PDF)
HISTOGRAM_BOXPLOT_PLOTLY(REVENUE_OPTIMUM, XLABEL='Optimum Revenue', TITLE='Optimum Revenue', COLOR='lime')

if stop == 1:
    NIM_OPTIMUM = INCOME_STATEMENT(Xmin, Xmax, SIZE_PDF) # Optimum Net Income Margin
    print(f"Optimum Pessimistic Revenue: {Xmin:.2f}"); 
    print(f"Optimum Optimistic Revenue: {Xmax:.2f}");
    HISTOGRAM_BOXPLOT_PLOTLY(NIM_OPTIMUM, XLABEL='Optimum Net Income Margin (%)', TITLE='Optimum Net Income Margin', COLOR='purple')